In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Emission Factors

In [2]:
greet_age = pd.read_csv('emission_factors/EERE_scenarios_TS_GREET1.csv', usecols=['FINAL_series name', 'Fuel Category', 'Vehicle Class', 'Year', 'LC Phase', 'Metric', 'BAU', 'Elec0', 'Elec_BAU'])
greet_eere = pd.read_csv('emission_factors/GREET_EF_EERE.csv', header=3, usecols=['GREET Pathway', 'Formula', 'Case', 'Year', 'Reference case', 'Elec0'])
greet_sc = pd.read_csv('emission_factors/GREET_EF_SUPPLY_CHAIN.csv', header=3, usecols=['GREET Pathway', 'Formula', 'Year', 'GREET Default', 'Zero Elec Comb'])
greet_hv = pd.read_csv('emission_factors/GREETheatingValues.csv', usecols=['GREET_Fuel', 'LHV_by_HHV'])
greet_mapping = pd.read_excel('emission_factors/GREET_CANOE_mapping.xlsx', sheet_name=['AGE_scenarios', 'GREET_EERE', 'GREET_SC', 'GREET_HV'])

In [3]:
greet_sc = greet_sc[(greet_sc.Year == 2022) | 
                    (greet_sc.Year == 2025) |
                    (greet_sc.Year == 2030) | 
                    (greet_sc.Year == 2035) | 
                    (greet_sc.Year == 2040) | 
                    (greet_sc.Year == 2045) |
                    (greet_sc.Year == 2050)]

greet_sc['fuel'] = greet_sc['GREET Pathway'].map(greet_mapping['GREET_SC'].set_index('GREET')['CANOE'])
greet_sc = greet_sc.sort_values(by='fuel')
greet_sc = greet_sc[greet_sc['GREET Default'] != 0]

greet_eere = greet_eere[(greet_eere.Year == 2022) | 
                        (greet_eere.Year == 2025) |
                        (greet_eere.Year == 2030) | 
                        (greet_eere.Year == 2035) | 
                        (greet_eere.Year == 2040) | 
                        (greet_eere.Year == 2045) |
                        (greet_eere.Year == 2050)]

greet_eere['fuel'] = greet_eere['GREET Pathway'].map(greet_mapping['GREET_EERE'].set_index('GREET')['CANOE'])
greet_eere = greet_eere.dropna(subset='fuel')
greet_eere = greet_eere.sort_values(by='fuel')
greet_eere = greet_eere[greet_eere['Reference case'] != 0]

greet_age = greet_age[(greet_age.Year == 2022) | 
                      (greet_age.Year == 2025) |
                      (greet_age.Year == 2030) | 
                      (greet_age.Year == 2035) | 
                      (greet_age.Year == 2040) | 
                      (greet_age.Year == 2045) |
                      (greet_age.Year == 2050)]

greet_age['fuel'] = greet_age['FINAL_series name'].map(greet_mapping['AGE_scenarios'].set_index('GREET')['CANOE'])
greet_age = greet_age.dropna(subset='fuel')
greet_age = greet_age.sort_values(by='fuel')
greet_age = greet_age[greet_age['BAU'] != 0]

greet_hv = greet_hv.dropna(subset='LHV_by_HHV')
greet_hv = greet_hv.drop_duplicates()
greet_hv['fuel'] = greet_hv['GREET_Fuel'].map(greet_mapping['GREET_HV'].dropna(subset='CANOE').set_index('GREET')['CANOE'])
greet_hv = greet_hv.dropna(subset='fuel')
greet_hv = greet_hv.sort_values(by='fuel')

In [4]:
new_rows = pd.DataFrame([{'GREET_Fuel': 'Marine Diesel Oil', 'LHV_by_HHV': 0.935, 'fuel': 'MDO'},
                         {'GREET_Fuel': 'Heavy Fuel Oil', 'LHV_by_HHV': 0.935, 'fuel': 'HFO'},
                         {'GREET_Fuel': 'CNG', 'LHV_by_HHV': 0.9, 'fuel': 'CNG'}])

greet_hv = pd.concat([greet_hv, new_rows], ignore_index=True)

greet_hv

,GREET_Fuel,LHV_by_HHV,fuel
0,"Methyl ester (biodiesel, BD)",0.934,BDSL
1,Coal Mix for Electricity Generation,0.942,COAL
2,Pet Coke,0.942,COKE
3,Low-sulfur diesel,0.935,DSL
4,Fischer-Tropsch diesel (FTD),0.951,EDSL
5,Ethanol,0.903,ETH
6,Gaseous hydrogen,0.845,GH2
7,Gasoline,0.932,GSL
8,Conventional Jet Fuel,0.935,JTF
9,Liquid hydrogen,0.847,LH2


# Supply chain

In [5]:
fig = px.scatter(greet_sc[(greet_sc.Year == 2022) & (greet_sc['Formula'] == 'CO2')], x='fuel', y='GREET Default', 
                 color='GREET Pathway', title='GREET WTP')

fig.show()

In [6]:
fig = px.scatter(greet_age[(greet_age.Year == 2022) & (greet_age['Metric'] == 'CO2') & (greet_age['LC Phase'] == 'Supply Chain')], x='fuel', y='BAU', 
                 color='Fuel Category', title='AGE GREET WTP')

fig.show()

In [7]:
greet_sc_export = greet_sc[greet_sc['Year'] == 2022][['fuel', 'GREET Pathway', 'Formula', 'GREET Default']]
greet_sc_export = pd.merge(greet_sc_export, greet_hv[['fuel', 'LHV_by_HHV']], how='left', on='fuel')
greet_sc_export['EF'] = greet_sc_export['GREET Default'] * greet_sc_export['LHV_by_HHV'] / 1055.05585 # MMBtu to MJ
greet_sc_export.drop(columns=['GREET Default', 'LHV_by_HHV'], inplace=True)
# greet_sc_export.pivot_table(index=['fuel', 'GREET Pathway'], columns='Formula', values='EF', aggfunc='first').reset_index()[['fuel', 'GREET Pathway', 'CO2', 'CH4', 'N2O']].to_clipboard()

greet_age_export = greet_age[(greet_age['Year'] == 2022) & (greet_age['LC Phase'] == 'Supply Chain')][['fuel', 'Fuel Category', 'Metric', 'BAU']]
greet_age_export = pd.merge(greet_age_export, greet_hv[['fuel', 'LHV_by_HHV']], how='left', on='fuel')
greet_age_export['EF'] = greet_age_export['BAU'] * greet_age_export['LHV_by_HHV'] / 1055.05585 # MMBtu to MJ
greet_age_export.drop(columns=['BAU', 'LHV_by_HHV'], inplace=True)
# greet_age_export.pivot_table(index=['fuel', 'Fuel Category'], columns='Metric', values='EF', aggfunc='first').reset_index()[['fuel', 'Fuel Category', 'CO2', 'CH4', 'N2O']].to_clipboard()

# Direct combustion

In [8]:
fig = px.scatter(greet_eere[(greet_eere.Year == 2022) & (greet_eere['Formula'] == 'CO2')], x='fuel', y='Reference case', 
                 color='GREET Pathway', title='GREET PTW')

fig.show()

In [9]:
fig = px.scatter(greet_age[(greet_age.Year == 2022) & (greet_age['Metric'] == 'CO2') & (greet_age['LC Phase'] == 'Direct, Combustion')], x='fuel', y='BAU', 
                 color='Fuel Category', title='AGE GREET PTW')

fig.show()

In [10]:
greet_eere_export = greet_eere[greet_eere['Year'] == 2022][['fuel', 'GREET Pathway', 'Formula', 'Reference case']]
greet_eere_export = pd.merge(greet_eere_export, greet_hv[['fuel', 'LHV_by_HHV']], how='left', on='fuel')
greet_eere_export['EF'] = greet_eere_export['Reference case'] * greet_eere_export['LHV_by_HHV'] / 1055.05585 # MMBtu to MJ
greet_eere_export.drop(columns=['Reference case', 'LHV_by_HHV'], inplace=True)
# greet_eere_export.pivot_table(index=['fuel', 'GREET Pathway'], columns='Formula', values='EF', aggfunc='first').reset_index()[['fuel', 'GREET Pathway', 'CO2', 'CH4', 'N2O']].to_clipboard()

greet_age_export = greet_age[(greet_age['Year'] == 2022) & (greet_age['LC Phase'] == 'Direct, Combustion')][['fuel', 'Vehicle Class', 'Metric', 'BAU']]
greet_age_export = pd.merge(greet_age_export, greet_hv[['fuel', 'LHV_by_HHV']], how='left', on='fuel')
greet_age_export['EF'] = greet_age_export['BAU'] * greet_age_export['LHV_by_HHV'] / 1055.05585 # MMBtu to MJ
greet_age_export.drop(columns=['BAU', 'LHV_by_HHV'], inplace=True)
# greet_age_export.pivot_table(index=['fuel', 'Vehicle Class'], columns='Metric', values='EF', aggfunc='first').reset_index()[['fuel', 'Vehicle Class', 'CO2', 'CH4', 'N2O']].to_clipboard()

# Fuel Prices

In [40]:
eth_data = pd.read_excel('fuel_prices/Biofuels-in-Canada-Analysis-FINAL-2023-10-31.xlsx', sheet_name='Gasoline Cost Analysis', header=2)
rdsl_data = pd.read_excel('fuel_prices/Biofuels-in-Canada-Analysis-FINAL-2023-10-31.xlsx', sheet_name='Diesel Cost Analysis', header=2, nrows=1250)

In [44]:
on_eth = eth_data[(eth_data.Region == 'Ontario')].drop(['Region', 'Month', 'Fiscal_Year'], axis=1)
on_eth = on_eth.groupby('Year', as_index=False).mean()

on_rdsl = rdsl_data[(rdsl_data.Region == 'Ontario')].drop(['Region', 'Month', 'Fiscal Year'], axis=1)
on_rdsl = on_rdsl.groupby('Year', as_index=False).mean()


In [48]:
fig = px.line(on_eth, x='Year', y=['Eth_P', 'BOB_P'])

fig.show()

In [49]:
fig = px.line(on_rdsl, x='Year', y=['BD_P', 'HDRD_P', 'Pure_Die_P'])

fig.show()